In [1]:
import pandas as pd
import matplotlib as plt

In [2]:
dataRead = pd.read_csv('aprobados_reprobados.csv')
data = dataRead.drop(['Nota primera oportunidad','Nota segunda oportunidad',],axis=1)

,Unnamed: 0,ID,Session 2,Session 3,Session 4,Session 5,Session 6,Nota primera oportunidad,Nota segunda oportunidad,aprobar,notaMax,intento
0,0,1,5.0,0.0,4.5,4.0,2.25,0.0,94.5,1,94.5,2
1,1,2,4.0,3.5,4.5,4.0,1.00,0.0,44.0,0,44.0,2
2,2,3,3.5,3.5,4.5,4.0,0.00,85.0,0.0,1,85.0,1
3,3,4,6.0,4.0,5.0,3.5,2.75,0.0,30.0,0,30.0,2
4,4,5,5.0,4.0,5.0,4.0,2.75,0.0,38.5,0,38.5,2


In [9]:
dataRead = pd.read_csv('aprobados_reprobados.csv')
df=dataRead[["ID","Session 2","Session 3","Session 4","Session 5","Session 6",'Nota primera oportunidad','Nota segunda oportunidad','notaMax','intento','aprobar']]
df.to_csv('aprobados_reprobados1.csv', index=False, header=True,encoding='utf-8')